<a href="https://colab.research.google.com/github/Mustafa-khattak/ML-Internship-2025/blob/main/Internship_Support_Chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class InternshipChatbot:
    def __init__(self):
        self.setup_knowledge_base()
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.questions)

    def setup_knowledge_base(self):
        self.faq_data = [

            {
                "category": "Application Process",
                "question": "How do I apply for an internship?",
                "answer": "You can apply for an internship through our official portal by submitting your resume, cover letter, and academic transcripts.",
                "keywords": ["apply", "application", "process", "resume", "cover letter"]
            },
            {
                "category": "Application Process",
                "question": "What documents do I need for internship application?",
                "answer": "You need a resume, cover letter, academic transcripts, and optionally recommendation letters from professors.",
                "keywords": ["documents", "resume", "transcript", "cover letter"]
            },

            {
                "category": "Interview",
                "question": "What should I expect in the internship interview?",
                "answer": "The interview typically includes behavioral and technical questions. Be prepared to discuss your academic background, projects, and motivation.",
                "keywords": ["interview", "questions", "prepare", "test", "assessment"]
            },


            {
                "category": "Certification",
                "question": "Will I receive a certificate after completing the internship?",
                "answer": "Yes, all interns receive an official Internship Completion Certificate, signed by the program coordinator.",
                "keywords": ["certificate", "completion", "proof", "recognition"]
            },


            {
                "category": "Networking",
                "question": "Can I connect with other interns and employees?",
                "answer": "Absolutely! You will have access to intern cohort activities, professional groups, mentorship programs, and social events.",
                "keywords": ["networking", "connect", "employees", "team", "friends"]
            },

            {
                "category": "Work-Life Balance",
                "question": "What about work-life balance during the internship?",
                "answer": "Interns usually work standard office hours (9–5) with flexibility if needed. We also encourage wellness and personal time off.",
                "keywords": ["work-life", "balance", "hours", "flexibility", "time off"]
            },

            {
                "category": "Stipend",
                "question": "Is the internship paid or unpaid?",
                "answer": "Some internships are paid depending on the program and role, while others are unpaid but provide valuable experience and certification.",
                "keywords": ["stipend", "salary", "paid", "unpaid", "money"]
            },
            {
                "category": "Remote Work",
                "question": "Can I do the internship remotely?",
                "answer": "Yes, we offer both on-site and remote internship options. Remote interns collaborate through online meetings and project tools.",
                "keywords": ["remote", "online", "work from home", "virtual"]
            },
            {
                "category": "Duration",
                "question": "How long does the internship last?",
                "answer": "Most internships last between 6 to 12 weeks, depending on the role and department needs.",
                "keywords": ["duration", "length", "weeks", "months"]
            },
            {
                "category": "Supervisor Support",
                "question": "Will I have a mentor or supervisor?",
                "answer": "Yes, each intern is assigned a supervisor or mentor who guides them, provides feedback, and helps with professional development.",
                "keywords": ["mentor", "supervisor", "guide", "support"]
            },
            {
                "category": "Skills Gained",
                "question": "What kind of skills will I gain?",
                "answer": "You will gain technical, communication, teamwork, and problem-solving skills that are valuable for your career.",
                "keywords": ["skills", "learning", "experience", "knowledge"]
            },
            {
                "category": "Future Opportunities",
                "question": "Can the internship lead to a full-time job?",
                "answer": "Yes, outstanding interns are often considered for future job opportunities or extended internships.",
                "keywords": ["future", "job", "opportunity", "career"]
            }
        ]

        self.knowledge_df = pd.DataFrame(self.faq_data)
        self.questions = [item['question'] for item in self.faq_data]
        self.answers = [item['answer'] for item in self.faq_data]
        self.all_keywords = []
        for item in self.faq_data:
            self.all_keywords.extend(item['keywords'])

    def find_best_match(self, user_query):
        query_vec = self.vectorizer.transform([user_query])
        similarities = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        best_idx = similarities.argmax()
        best_score = similarities[best_idx]
        return best_idx, best_score

    def generate_response(self, user_query):
        if not user_query.strip():
            return "Hello! I'm here to help with your internship questions. What would you like to know?"

        # If user requests all FAQs
        if user_query.lower() in ["all", "list", "help", "faq", "show all"]:
            response_lines = []
            for item in self.faq_data:
                response_lines.append(
                    f"**{item['category']}**\n\n**Q:** {item['question']}\n\n**A:** {item['answer']}\n"
                )
            return "\n---\n".join(response_lines)

        # Else: return the best match
        match_idx, similarity = self.find_best_match(user_query)
        if similarity > 0.15:
            faq_item = self.faq_data[match_idx]
            return f"**{faq_item['category']}**\n\n**Q:** {faq_item['question']}\n\n**A:** {faq_item['answer']}"
        else:
            return "I'm not sure about that. Try asking something like 'How do I apply?' or type 'all' to see every FAQ."

# ========================
# Gradio Interface
# ========================

chatbot = InternshipChatbot()

def chat_interface(user_message, history):
    bot_reply = chatbot.generate_response(user_message)
    history.append(("You", user_message))
    history.append(("Bot", bot_reply))
    return history, ""

with gr.Blocks(title="Internship Support Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Internship Support Chatbot")
    gr.Markdown("*Powered by offline AI - No downloads, no authentication errors!*")

    chatbot_interface = gr.Chatbot(
        value=[("System", "Hello! I'm your internship support assistant. What would you like to know?")],
        label="Chat with Internship Support Bot",
        height=500,
        show_label=True
    )

    msg = gr.Textbox(
        label="Your Message",
        placeholder="Type your internship question here...",
        lines=2
    )

    msg.submit(chat_interface, [msg, chatbot_interface], [chatbot_interface, msg])

    gr.Examples(
        examples=chatbot.questions,
        inputs=msg,
        label="Example Questions - Click to Try!"
    )

    gr.Markdown("---")
    gr.Markdown("### 📋 Full List of Available Questions:")
    gr.Markdown("\n".join([f"- {q}" for q in chatbot.questions]))

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cdd425a2a01c9185ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
